In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import sklearn
from omnixai.data.tabular import Tabular
from omnixai.preprocessing.tabular import TabularTransform
from omnixai.explainers.tabular import TabularExplainer
import shap
import joblib

Assuming model, test_data, test_labels, and original_data are already defined

Instantiate and fit the transformer

In [35]:

model = joblib.load('xgb_model.pkl')
df_private = joblib.load('df_private.pkl')
# Ensure the data types are correct

tabular_data = Tabular(
    df_private,
    categorical_columns=[
        "education-num", "race", "sex", "capital-gain", "capital-loss", "hours-per-week",
        "workclass=Federal-gov", "workclass=Local-gov", "workclass=Private",
        "workclass=Self-emp-inc", "native-country=Taiwan", "native-country=Thailand",
        "native-country=Trinadad&Tobago", "native-country=United-States",
        "native-country=Vietnam", "native-country=Yugoslavia", "age_binary", "sex_private"
    ],
    target_column='income-per-year'
)
transformer = TabularTransform().fit(tabular_data)
x = transformer.transform(tabular_data)
train, test, train_labels, test_labels = \
    sklearn.model_selection.train_test_split(x[:, :-1], x[:, -1], train_size=0.70,random_state = 1234)
train_data = transformer.invert(train)


Step 1: Identify misclassified and confident instances

In [36]:
selected_features = ["education-num", "race", "sex", "capital-gain", "capital-loss", "hours-per-week"]
train_data_selected = train_data[selected_features]
explainers = TabularExplainer(
    explainers=["ale", "shap_global"],                 # The explainers to apply
    mode="classification",                             # The task type
    data=train_data_selected,                                   # The data for initializing the explainers
    model=model,                                       # The ML model to explain
    preprocess=lambda z: transformer.transform(z[selected_features])  # Converts raw features into the model inputs
)
global_explanations = explainers.explain_global()


ValueError: invalid literal for int() with base 10: 'education-num'

In [ ]:
print("ALE results:")
global_explanations["ale"].ipython_plot(class_names=transformer.class_names)

In [ ]:
print("SHAP results:")
global_explanations["shap_global"].ipython_plot(class_names=transformer.class_names)

In [ ]:
def get_confident_misclassifications(model, test_data, test_labels, threshold=0.9):
    predictions_proba = model.predict_proba(test_data)  # Probabilities for each class
    predictions = model.predict(test_data)
    confidence = np.max(predictions_proba, axis=1)
    misclassified = (predictions != test_labels)
    high_confidence = confidence > threshold
    indices = np.where(misclassified & high_confidence)[0]
    return test_data[indices], test_labels[indices], predictions[indices], confidence[indices]

Step 2: Explain misclassifications using SHAP and LIME

In [ ]:
def explain_with_shap_and_lime(model, data_instance, transformer, test_data):
    # SHAP explanation
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(data_instance)
    print("SHAP values:")
    shap.initjs()
    shap.force_plot(explainer.expected_value[1], shap_values[1], data_instance)

    # LIME explanation
    explainers = TabularExplainer(
        explainers=['lime'],
        mode="classification",
        data=test_data,
        model=model,
        preprocess=lambda z: transformer.transform(z)
    )
    lime_explanation = explainers.explain(data_instance)
    lime_explanation['lime'].ipython_plot(index=0)

Step 3: Analyze the impact of noisy attributes


In [ ]:
def compare_noisy_and_original_attributes(misclassified_data, original_data):
    noisy_attributes = misclassified_data[['Age', 'Sex']]
    original_attributes = original_data.loc[misclassified_data.index, ['Age', 'Sex']]
    difference = noisy_attributes - original_attributes
    print("Differences in noisy vs original attributes:")
    print(difference)

Example usage :
Get confident misclassified instances

In [ ]:
confident_misclassified_data, confident_misclassified_labels, confident_predictions, confidences = get_confident_misclassifications(
    model, test_data, test_labels
)

Explain a misclassified instance

In [ ]:
if len(confident_misclassified_data) > 0:
    instance_id = 0  # Select the first misclassified instance
    instance_data = confident_misclassified_data[instance_id:instance_id + 1]
    explain_with_shap_and_lime(model, instance_data, transformer, test_data)

    # Compare noisy and original attributes for the misclassified instance
    compare_noisy_and_original_attributes(
        confident_misclassified_data.iloc[instance_id:instance_id + 1],
        original_data
    )
else:
    print("No confident misclassified instances found.")